# Welcome to the first notebook!

In this session we will work on a real-world dataset related to prediction of house price in a district.

The first thing you will do is to know how to run the code... see the `Run` button above?

You can also use the shortcut Ctrl + Enter.

Below is the first block of Python code to run - do not worry about what it means, these are mostly utilities and importing useful packages.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
rnd = 1
np.random.seed(rnd)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "1"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

print("Hello World!")

# 1. Prepare data

We will use a dataset of California census data 1990.

In [ ]:
HOUSING_PATH = os.path.join("data", "housing")

In [ ]:
# Load data
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)
housing = load_housing_data()

OK, the data is loaded, which is named `housing`. `housing` is a Pandas DataFrame object which resembles your (familiar) Excel sheet.  

Let's look at the first 5 rows of data with the `head` method:

In [ ]:
housing.head()

`Pandas` provides a lot of methods to interact and summarize data. For example, we can check on the basic statistics of each feature using the `describe` method:

In [ ]:
housing.describe(include='all')

Nothing speaks more than a beautiful picture. `Pandas` provides some plotting methods, but there are a lot of Python packages specialised on drawing graphs, for example `matplotlib`, `seaborn`, `plotly` and `bokeh`, just to name a few.

Let's start with something simple: since we have the Latitude and Longitude, what about a plot on the map?

In [ ]:
# plot 3D
# each dot is one sample on map, size of a dot proportional to population and color represents media house price.
# you may try to overlay an actual geographic map below the points!

housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
s=housing["population"]/100, label="population", figsize=(15,10),
c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
) 
plt.legend()

### Warning - you should NOT do this!
In a real machine learning project, we should plot a lot and a lot of graphs to look for abnormality, understand the physical meaning, gain insight, and develop questions etc. But for the sake of the moment, we will jump into modeling and come back later.
### End of Warning

For simplicity, we will only use 1,000 rows of the data (i.e. 1,000 samples).

Again, let's show the top 5 rows:

In [ ]:
# Select a subset of 1,000 samples
housing = housing.sample(n=1000, random_state=np.random.seed(rnd))
housing.head()

Another quick way to look for missing value: the `info` method:

In [ ]:
housing.info()

In [ ]:
housing.describe(include='all')

As we see, the dataset contains 1K rows (samples) with 10 attributes. 

Note there is one categorical variable `ocean_proximity`. (scroll to the right if you cannot see it).

## In this exercise, we will build a model to predict the `median_house_value`.

Let's start by plotting a histogram of our 'target': `median_house_value`. There is a `hist` method that comes in handy:

In [ ]:
housing['median_house_value'].hist(bins=50)  # 50 bins

Notice the suspicious spike near $500K? In practice we should go back to the data soucrce and look for the reason. It may even redefine our goal and problem!

But for now, let's simply filter them out to simplifiy the problem.

In [ ]:
# filter out samples with median_house_value >= 500,000
housing_flt = housing[housing['median_house_value'] < 500000]

In [ ]:
housing_flt['median_house_value'].hist(bins=50)

Yet another handy method (actually a `property`, to be precise) to get the shape of data: `shape`

In [ ]:
# We have filtered out 35 (1000-965) records
housing_flt.shape

In [ ]:
housing_flt.describe(include='all')

...how about listing the linear correlation between our target and each feature?

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

#### Reflection: does the correlation make sense?

Now we have the data, what's next?

Before we dive in, there is a very important step to do: let's split the data set into 2, one for training the model (commonly called the * *training set* *) and the other for evaluation (the * *test set* *).

This is **VERY IMPORTANT** in modeling - we need to hold out a set of data, which is not known by the model, to evaluate how well the model perform.

An analogue, in school - we are *__trained__* with materials in lessons (reading, lectures...). To evaluate how well we understand the materials, we are *__tested__* in with materials that are relevant, but not known in advance (i.e. examination). Otherwise we will simply recite (* *memorize* *) the questions and answers in exam and get a good score, without actually understand the materials at all!

### Question: what problem will it pose if a model has access to the test data during training?
Hint: you will know in coming lecture!

In [ ]:
# Split the data into train and test set (90:10 split)
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing_flt, test_size=0.1)

# 2. Train the first model

OK, time to begin modeling!

As a start, let's use the only 1 feature, `median_income`, to predict our target (label), `median_house_value`.

In [ ]:
# Use only 2 columns, "median_income" and "median_house_value"
usecols = ['median_income', 'median_house_value']
train_set_univariate = train_set[usecols]
test_set_univariate = test_set[usecols]

In [ ]:
# Draw a plot first...
train_set_univariate.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

OK, the relationship looks quite linear... (expected, it is what our correlation table above told us). 

Therefore, how about we start with one of the most common and simplest model, the *Univariate Linear Regression*?

### Question: what is the formula of a simple (univariate) linear model?

We will use a Python package, Scikit-Learn `sklearn`, for the model training etc..

In [ ]:
# import the Linear Regression model
from sklearn.linear_model import LinearRegression
# import an error metric (in this case the mean squared error, MSE)
from sklearn.metrics import mean_squared_error

train_set_univariate__X = np.array(train_set_univariate['median_income']).reshape(-1, 1)  # preparing feature (data X)
train_set_univariate__y = train_set_univariate['median_house_value']  # preparing the target (y)

linear_reg = LinearRegression()
linear_reg.fit(X=train_set_univariate__X, y=train_set_univariate__y)  # feed X and y into the linear model, ask it to 'train'

The model is trained. (Yes, done!)

Let's show its parameters:

In [ ]:
print("Slope `m`: {:,.2f}\nIntercept `c`: {:,.2f}".format(linear_reg.coef_[0], linear_reg.intercept_))

### Question: what is the formula of the trained model?

Time to test how well our model is doing - as the first thing, how about applying this model on the training data, get the predictions, and draw some graphs?

### Warning: this is NOT the correct way to evaluate a model - we should use a validation or test set. This is only for sake of illusration.

In [ ]:
# get predictions for train set
train_set_univariate__y_pred = linear_reg.predict(train_set_univariate__X)

In [ ]:
# plot a grahp
train_set_univariate.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.4)
plt.plot(train_set_univariate__X, train_set_univariate__y_pred, color='red', linewidth=2)

In [ ]:
# evaluate
from sklearn import metrics

def print_eval(y_true, y_pred):
    print('Average target in train set: {:,.0f}'.format(y_true.mean()))
    print('Mean Absolute Error on train set: {:,.0f}'.format(metrics.mean_absolute_error(y_true, y_pred)))
    print('Root Mean Squared Error on train set: {:,.0f}'.format(np.sqrt(metrics.mean_squared_error(y_true, y_pred))))
    print('Relative Root Mean Squared Error on train set: {:.2f} %'.format(100 * np.sqrt(metrics.mean_squared_error(y_true, y_pred))/y_true.mean()))
    print('R^2 of train set: {:,.2f}'.format(metrics.r2_score(y_true, y_pred)))
    return None

print_eval(train_set_univariate__y, train_set_univariate__y_pred)

## Exercise 1
In the example above, the predictions (red line) seems skewed towards clockwise direction, probably due to the few data points with high `median_income`.

Let's assume the data points with `median_income` > 9 are outlier, repeat the modeling process (prepare data, train model) above and draw a graph of the result.

# 3. Improve model performance

The performace is not great - an RMSE of ~70K to a target of ~200K is a bit too high, even on train set...

What can we do to improve?

There are a few things we can try:

1. Collect more samples
2. Try a different combination of features
3. Try a different models

Let's try #2 and use more features!

In [ ]:
# Try adding a new feature, 'households'
usecols = ['median_income', 'housing_median_age', 'median_house_value']
from pandas.plotting import scatter_matrix
scatter_matrix(housing_flt[usecols], alpha=0.2, figsize=(15, 12), diagonal='hist')

In [ ]:
# Again, prepare train and test sets
train_set_bivariates = train_set[usecols]
test_set_bivariates = test_set[usecols]

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

train_set_bivariates__X = train_set_bivariates[['median_income', 'housing_median_age']]
train_set_bivariates__y = train_set_bivariates['median_house_value']

linear_reg_2 = LinearRegression()
linear_reg_2.fit(X=train_set_bivariates__X, y=train_set_bivariates__y)

# get predictions
train_set_bivariates__pred = linear_reg_2.predict(train_set_bivariates__X)

In [ ]:
print("Slopes m1 and m2: {:,.2f}, {:,.2f}\nIntercept: {:,.2f}".format(linear_reg_2.coef_[0], linear_reg_2.coef_[1], linear_reg_2.intercept_))

In [ ]:
print_eval(train_set_bivariates__y, train_set_bivariates__pred)